## 0. Imports, libraries and rusable functions

In [1]:
# Standard Library Imports
import ast
import copy
import csv
import json
import math
import os
import re
import time
import warnings
import logging
import random
import collections
from collections import Counter
from typing import List, Tuple, Optional
from IPython.display import HTML, display
import math
import time
from unidecode import unidecode


# Data Handling Libraries
import numpy as np
import pandas as pd
import csv
from torch.utils.data import random_split
import datasets
from datasets import ClassLabel, Sequence

# Data Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
# import scikitplot as skplt  # Uncomment if scikit-plot is installed and needed

# Machine Learning: Model Preparation
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
from sklearn.model_selection import cross_val_score, cross_validate, KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler

# Machine Learning: Models and Frameworks
import tensorflow as tf
import torch
import evaluate
import xgboost
import wandb
from xgboost import plot_importance  # Uncomment if xgboost importance plot is required


# NLP and Transformers
from transformers import (AdamW, AutoModelForSequenceClassification, AutoModelForQuestionAnswering,
                          AutoTokenizer, CamembertForSequenceClassification, DistilBertConfig,
                          DistilBertForSequenceClassification, DistilBertModel, EarlyStoppingCallback,
                          get_linear_schedule_with_warmup, RobertaForSequenceClassification, EvalPrediction,
                          Trainer, TrainerCallback, TrainingArguments, XLMRobertaForSequenceClassification,
                         DefaultDataCollator, BertForQuestionAnswering, DataCollatorWithPadding, PreTrainedTokenizerFast,
                         default_data_collator, is_torch_xla_available)
from datasets import Dataset, DatasetDict, load_dataset
from transformers.trainer_utils import PredictionOutput, speed_metrics

# Experiment Tracking and Optimization Utilities
import optuna
from optuna.trial import TrialState
# import wandb  # Uncomment if using Weights & Biases for experiment tracking

# Progress Bar Utilities
from tqdm.notebook import tqdm


In [2]:
#wandb.login(key='8f7092f0fdaf14add2b4cc07cb0e740080cdd8e7')
wandb.login()

wandb: Currently logged in as: mzak071 (COMPSCI714). Use `wandb login --relogin` to force relogin


True

In [3]:
class LoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

GPU: NVIDIA GeForce RTX 4070 Ti SUPER is available.
cuda


## 1. Global Variables

In [6]:
## Arguments and global vriables
global_doc_stride = 128
pretrained_model_name = "FacebookAI/roberta-base"
normalized_model_name = pretrained_model_name.replace("/", "-")
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
assert isinstance( tokenizer, PreTrainedTokenizerFast )
data_collator = DefaultDataCollator()
max_seq_length = tokenizer.model_max_length
version_2_with_negative = True
no_answer_threshold = 0.0166
right_padding = tokenizer.padding_side == 'right'
global_counter = 0
traing_answer_mismatches = []
logger = logging.getLogger(__name__)
global train_dataset
global eval_dataset
global cleaned_training_dataset

## 1. Q&A Reusable Functions

In [8]:
### Compute_metrics function for Question and Answering problem is different to classification, more preocessing required.
metric = evaluate.load("squad_v2")

def compute_metrics(p: EvalPrediction):
        return metric.compute(predictions=p.predictions, references=p.label_ids)

In [9]:
"""
A subclass of `Trainer` specific to Question-Answering tasks
"""

if is_torch_xla_available():
    import torch_xla.core.xla_model as xm
    import torch_xla.debug.metrics as met


class QuestionAnsweringTrainer(Trainer):
    def __init__(self, *args, eval_examples=None, post_process_function=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.eval_examples = eval_examples
        self.post_process_function = post_process_function

    def evaluate(self, eval_dataset=None, eval_examples=None, ignore_keys=None, metric_key_prefix: str = "eval"):
        eval_dataset = self.eval_dataset if eval_dataset is None else eval_dataset
        eval_dataloader = self.get_eval_dataloader(eval_dataset)
        eval_examples = self.eval_examples if eval_examples is None else eval_examples

        # Temporarily disable metric computation, we will do it in the loop here.
        compute_metrics = self.compute_metrics
        self.compute_metrics = None
        eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else self.evaluation_loop
        start_time = time.time()
        try:
            output = eval_loop(
                eval_dataloader,
                description="Evaluation",
                # No point gathering the predictions if there are no metrics, otherwise we defer to
                # self.args.prediction_loss_only
                prediction_loss_only=True if compute_metrics is None else None,
                ignore_keys=ignore_keys,
                metric_key_prefix=metric_key_prefix,
            )
        finally:
            self.compute_metrics = compute_metrics
        total_batch_size = self.args.eval_batch_size * self.args.world_size
        if f"{metric_key_prefix}_jit_compilation_time" in output.metrics:
            start_time += output.metrics[f"{metric_key_prefix}_jit_compilation_time"]
        output.metrics.update(
            speed_metrics(
                metric_key_prefix,
                start_time,
                num_samples=output.num_samples,
                num_steps=math.ceil(output.num_samples / total_batch_size),
            )
        )
        if self.post_process_function is not None and self.compute_metrics is not None and self.args.should_save:
            # Only the main node write the results by default
            eval_preds = self.post_process_function(eval_examples, eval_dataset, output.predictions)
            metrics = self.compute_metrics(eval_preds)

            # Prefix all keys with metric_key_prefix + '_'
            for key in list(metrics.keys()):
                if not key.startswith(f"{metric_key_prefix}_"):
                    metrics[f"{metric_key_prefix}_{key}"] = metrics.pop(key)
            metrics.update(output.metrics)
        else:
            metrics = output.metrics

        if self.args.should_log:
            # Only the main node log the results by default
            self.log(metrics)

        if self.args.tpu_metrics_debug or self.args.debug:
            # tpu-comment: Logging debug metrics for PyTorch/XLA (compile, execute times, ops, etc.)
            xm.master_print(met.metrics_report())

        self.control = self.callback_handler.on_evaluate(self.args, self.state, self.control, metrics)
        return metrics

    def predict(self, predict_dataset, predict_examples, ignore_keys=None, metric_key_prefix: str = "test"):
        predict_dataloader = self.get_test_dataloader(predict_dataset)

        # Temporarily disable metric computation, we will do it in the loop here.
        compute_metrics = self.compute_metrics
        self.compute_metrics = None
        eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else self.evaluation_loop
        start_time = time.time()
        try:
            output = eval_loop(
                predict_dataloader,
                description="Prediction",
                # No point gathering the predictions if there are no metrics, otherwise we defer to
                # self.args.prediction_loss_only
                prediction_loss_only=True if compute_metrics is None else None,
                ignore_keys=ignore_keys,
                metric_key_prefix=metric_key_prefix,
            )
        finally:
            self.compute_metrics = compute_metrics
        total_batch_size = self.args.eval_batch_size * self.args.world_size
        if f"{metric_key_prefix}_jit_compilation_time" in output.metrics:
            start_time += output.metrics[f"{metric_key_prefix}_jit_compilation_time"]
        output.metrics.update(
            speed_metrics(
                metric_key_prefix,
                start_time,
                num_samples=output.num_samples,
                num_steps=math.ceil(output.num_samples / total_batch_size),
            )
        )

        if self.post_process_function is None or self.compute_metrics is None:
            return output

        predictions = self.post_process_function(predict_examples, predict_dataset, output.predictions, "predict")
        metrics = self.compute_metrics(predictions)

        # Prefix all keys with metric_key_prefix + '_'
        for key in list(metrics.keys()):
            if not key.startswith(f"{metric_key_prefix}_"):
                metrics[f"{metric_key_prefix}_{key}"] = metrics.pop(key)
        metrics.update(output.metrics)
        return PredictionOutput(predictions=predictions.predictions, label_ids=predictions.label_ids, metrics=metrics)

In [12]:
def prepare_validation_features(examples):
  # Some of the questions have lots of whitespace on the left, which is not useful and will make the
  # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
  # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]
  # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
  # in one example possible giving several features when a context is long, each of those features having a
  # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
      examples["question"],
      examples["context"],
      truncation="only_second" if right_padding else "only_first",
      max_length=max_seq_length,
      stride=global_doc_stride,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length",
  )

  # Since one example might give us several features if it has a long context, we need a map from a feature to
  # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

  # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
  # corresponding example_id and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
      # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 0

      # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

      # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
      # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
      ]

    return tokenized_examples

In [14]:
# Validation preprocessing
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if right_padding else "context"],
        examples["context" if right_padding else "question"],
        truncation="only_second" if right_padding else "only_first",
        max_length=max_seq_length,
        stride=global_doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
    # corresponding example_id and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if right_padding else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [15]:
# Post-processing:
def post_processing_function(examples, features, predictions, stage="eval"):
    # Post-processing: we match the start logits and end logits to answers in the original context.
    predictions = postprocess_qa_predictions(
        examples=examples,
        features=features,
        predictions=predictions,
        version_2_with_negative=version_2_with_negative, #If true, some of the examples do not have an answer.
        n_best_size=20, #The total number of n-best predictions to generate when looking for an answer.
        max_answer_length=30, #The maximum length of an answer that can be generated. This is needed because the start and end predictions are not conditioned on one another.
        null_score_diff_threshold=no_answer_threshold, #The threshold used to select the null answer: if the best answer has a score that is less than the score of the null answer minus this threshold, the null answer is selected for this example. Only useful when `version_2_with_negative=True`.
        output_dir= None,
        log_level=logging.WARNING,
        prefix=stage,
    )
    # Format the result to the format the metric expects.
    if version_2_with_negative:
        formatted_predictions = [
                {"id": str(k), "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()
            ]
    else:
        formatted_predictions = [{"id": str(k), "prediction_text": v} for k, v in predictions.items()]
    
    
    references = [{"id": str(ex["id"]), "answers": ex["answers"]} for ex in examples]
    return EvalPrediction(predictions=formatted_predictions, label_ids=references)

In [16]:
def postprocess_qa_predictions(
    examples,
    features,
    predictions: Tuple[np.ndarray, np.ndarray],
    version_2_with_negative: bool = False,
    n_best_size: int = 20,
    max_answer_length: int = 30,
    null_score_diff_threshold: float = 0.0,
    output_dir: Optional[str] = None,
    prefix: Optional[str] = None,
    log_level: Optional[int] = logging.WARNING,
):
    """
    Post-processes the predictions of a question-answering model to convert them to answers that are substrings of the
    original contexts. This is the base postprocessing functions for models that only return start and end logits.

    Args:
        examples: The non-preprocessed dataset (see the main script for more information).
        features: The processed dataset (see the main script for more information).
        predictions (:obj:`Tuple[np.ndarray, np.ndarray]`):
            The predictions of the model: two arrays containing the start logits and the end logits respectively. Its
            first dimension must match the number of elements of :obj:`features`.
        version_2_with_negative (:obj:`bool`, `optional`, defaults to :obj:`False`):
            Whether or not the underlying dataset contains examples with no answers.
        n_best_size (:obj:`int`, `optional`, defaults to 20):
            The total number of n-best predictions to generate when looking for an answer.
        max_answer_length (:obj:`int`, `optional`, defaults to 30):
            The maximum length of an answer that can be generated. This is needed because the start and end predictions
            are not conditioned on one another.
        null_score_diff_threshold (:obj:`float`, `optional`, defaults to 0):
            The threshold used to select the null answer: if the best answer has a score that is less than the score of
            the null answer minus this threshold, the null answer is selected for this example (note that the score of
            the null answer for an example giving several features is the minimum of the scores for the null answer on
            each feature: all features must be aligned on the fact they `want` to predict a null answer).

            Only useful when :obj:`version_2_with_negative` is :obj:`True`.
        output_dir (:obj:`str`, `optional`):
            If provided, the dictionaries of predictions, n_best predictions (with their scores and logits) and, if
            :obj:`version_2_with_negative=True`, the dictionary of the scores differences between best and null
            answers, are saved in `output_dir`.
        prefix (:obj:`str`, `optional`):
            If provided, the dictionaries mentioned above are saved with `prefix` added to their names.
        log_level (:obj:`int`, `optional`, defaults to ``logging.WARNING``):
            ``logging`` log level (e.g., ``logging.WARNING``)
    """
    if len(predictions) != 2:
        raise ValueError("`predictions` should be a tuple with two elements (start_logits, end_logits).")
    all_start_logits, all_end_logits = predictions

    if len(predictions[0]) != len(features):
        raise ValueError(f"Got {len(predictions[0])} predictions and {len(features)} features.")

    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    all_predictions = collections.OrderedDict()
    all_nbest_json = collections.OrderedDict()
    if version_2_with_negative:
        scores_diff_json = collections.OrderedDict()

    # Logging.
    logger.setLevel(log_level)
    logger.info(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_prediction = None
        prelim_predictions = []

        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]
            # Optional `token_is_max_context`, if provided we will remove answers that do not have the maximum context
            # available in the current feature.
            token_is_max_context = features[feature_index].get("token_is_max_context", None)

            # Update minimum null prediction.
            feature_null_score = start_logits[0] + end_logits[0]
            if min_null_prediction is None or min_null_prediction["score"] > feature_null_score:
                min_null_prediction = {
                    "offsets": (0, 0),
                    "score": feature_null_score,
                    "start_logit": start_logits[0],
                    "end_logit": end_logits[0],
                }

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or len(offset_mapping[start_index]) < 2
                        or offset_mapping[end_index] is None
                        or len(offset_mapping[end_index]) < 2
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue
                    # Don't consider answer that don't have the maximum context available (if such information is
                    # provided).
                    if token_is_max_context is not None and not token_is_max_context.get(str(start_index), False):
                        continue

                    prelim_predictions.append(
                        {
                            "offsets": (offset_mapping[start_index][0], offset_mapping[end_index][1]),
                            "score": start_logits[start_index] + end_logits[end_index],
                            "start_logit": start_logits[start_index],
                            "end_logit": end_logits[end_index],
                        }
                    )
        if version_2_with_negative and min_null_prediction is not None:
            # Add the minimum null prediction
            prelim_predictions.append(min_null_prediction)
            null_score = min_null_prediction["score"]

        # Only keep the best `n_best_size` predictions.
        predictions = sorted(prelim_predictions, key=lambda x: x["score"], reverse=True)[:n_best_size]

        # Add back the minimum null prediction if it was removed because of its low score.
        if (
            version_2_with_negative
            and min_null_prediction is not None
            and not any(p["offsets"] == (0, 0) for p in predictions)
        ):
            predictions.append(min_null_prediction)

        # Use the offsets to gather the answer text in the original context.
        context = example["context"]
        for pred in predictions:
            offsets = pred.pop("offsets")
            pred["text"] = context[offsets[0] : offsets[1]]

        # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
        # failure.
        if len(predictions) == 0 or (len(predictions) == 1 and predictions[0]["text"] == ""):
            predictions.insert(0, {"text": "empty", "start_logit": 0.0, "end_logit": 0.0, "score": 0.0})

        # Compute the softmax of all scores (we do it with numpy to stay independent from torch/tf in this file, using
        # the LogSumExp trick).
        scores = np.array([pred.pop("score") for pred in predictions])
        exp_scores = np.exp(scores - np.max(scores))
        probs = exp_scores / exp_scores.sum()

        # Include the probabilities in our predictions.
        for prob, pred in zip(probs, predictions):
            pred["probability"] = prob

        # Pick the best prediction. If the null answer is not possible, this is easy.
        if not version_2_with_negative:
            all_predictions[example["id"]] = predictions[0]["text"]
        else:
            # Otherwise we first need to find the best non-empty prediction.
            i = 0
            while predictions[i]["text"] == "":
                i += 1
            best_non_null_pred = predictions[i]

            # Then we compare to the null prediction using the threshold.
            score_diff = null_score - best_non_null_pred["start_logit"] - best_non_null_pred["end_logit"]
            scores_diff_json[example["id"]] = float(score_diff)  # To be JSON-serializable.
            if score_diff > null_score_diff_threshold:
                all_predictions[example["id"]] = ""
            else:
                all_predictions[example["id"]] = best_non_null_pred["text"]

        # Make `predictions` JSON-serializable by casting np.float back to float.
        all_nbest_json[example["id"]] = [
            {k: (float(v) if isinstance(v, (np.float16, np.float32, np.float64)) else v) for k, v in pred.items()}
            for pred in predictions
        ]

    # If we have an output_dir, let's save all those dicts.
    if output_dir is not None:
        if not os.path.isdir(output_dir):
            raise EnvironmentError(f"{output_dir} is not a directory.")

        prediction_file = os.path.join(
            output_dir, "predictions.json" if prefix is None else f"{prefix}_predictions.json"
        )
        nbest_file = os.path.join(
            output_dir, "nbest_predictions.json" if prefix is None else f"{prefix}_nbest_predictions.json"
        )
        if version_2_with_negative:
            null_odds_file = os.path.join(
                output_dir, "null_odds.json" if prefix is None else f"{prefix}_null_odds.json"
            )

        logger.info(f"Saving predictions to {prediction_file}.")
        with open(prediction_file, "w") as writer:
            writer.write(json.dumps(all_predictions, indent=4) + "\n")
        logger.info(f"Saving nbest_preds to {nbest_file}.")
        with open(nbest_file, "w") as writer:
            writer.write(json.dumps(all_nbest_json, indent=4) + "\n")
        if version_2_with_negative:
            logger.info(f"Saving null_odds to {null_odds_file}.")
            with open(null_odds_file, "w") as writer:
                writer.write(json.dumps(scores_diff_json, indent=4) + "\n")

    return all_predictions

In [18]:
# List of questions to debug
debug_questions = [
    "What ideology was sponsored at the Ming court?",
    "Who stopped their trips to Ming China?",
    "Another question of interest"
]

# Training preprocessing
def preprocess_function(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if right_padding else "context"],
        examples["context" if right_padding else "question"],
        truncation="only_second" if right_padding else "only_first",
        max_length=max_seq_length,
        stride=global_doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]        
        question = examples["question"][sample_index]
        example_id = examples["id"][sample_index]
        
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if right_padding else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if right_padding else 0):
                token_end_index -= 1
            
            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Increase the end index here to make sure it includes the last character
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                safe_start_index = max(0, token_start_index - 1)
                tokenized_examples["start_positions"].append(safe_start_index)

                # Adjust this line to ensure that the end character is included
                # Decrement token_end_index to point exactly one past the end of the answer text
                while token_end_index >= 0 and offsets[token_end_index][1] > end_char:
                    token_end_index -= 1
                safe_end_index = min(len(input_ids) - 1, token_end_index + 1)
                tokenized_examples["end_positions"].append(safe_end_index)  # Adjust to +2 if needed
                
        # Debugging code
        global global_counter
        global traing_answer_mismatches
        #if question in debug_questions:
        if len(answers["answer_start"]) > 0:
            # Ensure indices are within bounds
            #safe_start_index = max(0, token_start_index - 1)
            #safe_end_index = min(len(input_ids) - 1, token_end_index + 1)

            # Decode the answer using safe indices
            decoded_answer = tokenizer.decode(input_ids[safe_start_index:safe_end_index])
            actual_answer = answers["text"][0] if answers["text"] else "No answer provided"
            normalized_actual_answer = actual_answer.lower().replace(" ", "")
            normalized_decoded_answer = decoded_answer.lower().replace(" ", "")
            if normalized_decoded_answer != normalized_actual_answer:
                if global_counter == 0: print('Mismatch Found')
                global_counter += 1 
                traing_answer_mismatches.append({
                    'TokenizedId': i,
                    'ID': example_id,
                    'Question': question,
                    'Offsets': offsets,
                    'Input IDs':input_ids,
                    'Tokenized Text': tokenizer.decode(input_ids),
                    'Char start/end index': f"{start_char} / {end_char}",
                    'Token start/end index':f"{token_start_index} / {token_end_index}",
                    'Decoded Answer': decoded_answer,
                    'Actual Answer': actual_answer,                    
                })    
    return tokenized_examples

## 2. Data Preparation

In [26]:
# Load the full dataset
full_squad_dataset = load_dataset("squad_v2")

In [27]:
Use_Only_Sample = False
Sample_Size = 10000

if Use_Only_Sample:
    # Load a sample portion of the dataset
    squad_raw = datasets.DatasetDict({
        'train': full_squad_dataset['train'].select(range(0, Sample_Size)),
        'validation': full_squad_dataset['validation'].select(range(0, int(Sample_Size * 0.2)))
    })
else:
    # Load the full dataset
    squad_raw = datasets.DatasetDict({
        'train': full_squad_dataset['train'],
        'validation': full_squad_dataset['validation'],    
    })
    
# Display the sizes of the splits to confirm
print("Train set size:", len(squad_raw['train']))
print("Validation set size:", len(squad_raw['validation']))
squad_raw

Train set size: 130319
Validation set size: 11873


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [28]:
## Preprocess the Training Dataset To find Missmatches
global_counter = 0
traing_answer_mismatches = []

train_dataset = squad_raw['train'].map(
                preprocess_function,
                batched=True,
                remove_columns=squad_raw["train"].column_names,
                desc="Running tokenizer on train dataset",
                load_from_cache_file=False,  # Disable caching
            )


Running tokenizer on train dataset:   0%|          | 0/130319 [00:00<?, ? examples/s]

Mismatch Found


In [29]:
# Analyse Missmatches
mismatches_df = pd.DataFrame(traing_answer_mismatches)
print('Total number of miss matches:',global_counter)
display(mismatches_df.head(10))  

Total number of miss matches: 1409


,TokenizedId,ID,Question,Offsets,Input IDs,Tokenized Text,Char start/end index,Token start/end index,Decoded Answer,Actual Answer
0,77,56be8c8a3aeaaa14008c90ab,"For which song, did Destiny's Child take home ...","[(0, 0), (0, 3), (4, 9), (10, 14), (14, 15), (...","[0, 2709, 61, 2214, 6, 222, 23313, 18, 7442, 1...","<s>For which song, did Destiny's Child take ho...",848 / 861,228 / 230,"""Say My Name","""Say My Name"""
1,92,56bf7e603aeaaa14008c9681,What event caused Beyonce's depression?,"[(0, 0), (0, 4), (5, 10), (11, 17), (18, 23), ...","[0, 2264, 515, 1726, 12674, 1755, 18, 6943, 11...",<s>What event caused Beyonce's depression?</s>...,194 / 222,48 / 52,split with Luckett and Rob,split with Luckett and Rober
2,148,56d4baf92ccc5a1400d8317f,What is the name of the final studio album fro...,"[(0, 0), (0, 4), (5, 7), (8, 11), (12, 16), (1...","[0, 2264, 16, 5, 766, 9, 5, 507, 4535, 2642, 3...",<s>What is the name of the final studio album ...,848 / 866,200 / 202,Destiny Fulfilled,Destiny Fulfilled.
3,203,56be973d3aeaaa14008c9123,How many number one singles did Beyonce now ha...,"[(0, 0), (0, 3), (4, 8), (9, 15), (16, 19), (2...","[0, 6179, 171, 346, 65, 7695, 222, 12674, 1755...",<s>How many number one singles did Beyonce now...,457 / 460,128 / 126,,six
4,331,56d4d0c32ccc5a1400d8324e,In which music video did Beyoncé star as Jay Z...,"[(0, 0), (0, 2), (3, 8), (9, 14), (15, 20), (2...","[0, 1121, 61, 930, 569, 222, 12674, 12695, 999...",<s>In which music video did Beyoncé star as Ja...,94 / 112,42 / 45,"""'03 Bonnie & Clyde",'03 Bonnie & Clyde
5,354,56bfacdda10cfb140055122d,Who was credited for her cries on the song?,"[(0, 0), (0, 3), (4, 7), (8, 16), (17, 20), (2...","[0, 12375, 21, 11223, 13, 69, 25355, 15, 5, 22...",<s>Who was credited for her cries on the song?...,457 / 463,113 / 116,B.I.C,B.I.C.
6,359,56d4d2232ccc5a1400d83266,How was Blue Ivy credited on Glory?,"[(0, 0), (0, 3), (4, 7), (8, 12), (13, 16), (1...","[0, 6179, 21, 2692, 19647, 11223, 15, 30402, 1...",<s>How was Blue Ivy credited on Glory?</s></s>...,457 / 463,111 / 114,B.I.C,B.I.C.
7,506,56bec3ea3aeaaa14008c93a0,Her alter ego was born when according to Beyonce?,"[(0, 0), (0, 3), (4, 9), (10, 13), (14, 17), (...","[0, 13584, 11330, 21450, 21, 2421, 77, 309, 7,...",<s>Her alter ego was born when according to Be...,378 / 403,102 / 107,"making of ""Crazy in Love","making of ""Crazy in Love"""
8,817,56cbda8d6d243a140015ed8e,In what village was Frédéric born in?,"[(0, 0), (0, 2), (3, 7), (8, 15), (16, 19), (2...","[0, 1121, 99, 3375, 21, 4967, 1140, 417, 1140,...",<s>In what village was Frédéric born in?</s></...,28 / 41,26 / 30,�elazowa Wola,Żelazowa Wola
9,822,56ce1225aab44d1400b88430,Where was Chopin born?,"[(0, 0), (0, 5), (6, 9), (10, 14), (14, 16), (...","[0, 13841, 21, 15395, 179, 2421, 116, 2, 2, 59...",<s>Where was Chopin born?</s></s>Fryderyk Chop...,28 / 41,20 / 24,�elazowa Wola,Żelazowa Wola


In [30]:
# List of specific IDs to inspect
debug_ids = [
    "56be8c8a3aeaaa14008c90ab",       
]

# Define a function to filter examples by ID and stop processing when all have been found
def print_specific_records(dataset):
    found_ids = set()  # To track IDs that have been printed
    for example in dataset:
        if example['id'] in debug_ids:
            print(example)
            found_ids.add(example['id'])
            if found_ids == set(debug_ids):
                break  # Stop processing as all requested records have been printed

# Apply the function to the full dataset (assuming the dataset is loaded and named full_squad_dataset)
print_specific_records(full_squad_dataset['train'])


{'id': '56be8c8a3aeaaa14008c90ab', 'title': 'Beyoncé', 'context': 'The group changed their name to Destiny\'s Child in 1996, based upon a passage in the Book of Isaiah. In 1997, Destiny\'s Child released their major label debut song "Killing Time" on the soundtrack to the 1997 film, Men in Black. The following year, the group released their self-titled debut album, scoring their first major hit "No, No, No". The album established the group as a viable act in the music industry, with moderate sales and winning the group three Soul Train Lady of Soul Awards for Best R&B/Soul Album of the Year, Best R&B/Soul or Rap New Artist, and Best R&B/Soul Single for "No, No, No". The group released their multi-platinum second album The Writing\'s on the Wall in 1999. The record features some of the group\'s most widely known songs such as "Bills, Bills, Bills", the group\'s first number-one single, "Jumpin\' Jumpin\'" and "Say My Name", which became their most successful song at the time, and would 

In [31]:
# Ensure the mismatches DataFrame is created from non-empty data
if traing_answer_mismatches:
    mismatches_df = pd.DataFrame(traing_answer_mismatches)
    print('Total number of mismatches:', len(mismatches_df))
else:
    print("No mismatches to display. The list is empty.")

def remove_mismatches(dataset, mismatch_ids):
    """
    Filters the dataset to exclude mismatched entries.

    Args:
        dataset (Dataset): The dataset from which to remove mismatched examples.
        mismatch_ids (set): A set of example IDs that have mismatches.

    Returns:
        Dataset: A dataset with mismatched examples removed.
    """
    # Use a set for faster look-up times
    filtered_dataset = dataset.filter(lambda example: example['id'] not in mismatch_ids)
    return filtered_dataset

# Check if mismatches DataFrame exists and is not empty before removing mismatches
if 'traing_answer_mismatches' in locals() and not mismatches_df.empty:
    mismatch_ids = set(mismatches_df['ID'])
    cleaned_training_dataset = remove_mismatches(squad_raw['train'], mismatch_ids)
    print("Original dataset size:", len(squad_raw['train']))
    print("Cleaned dataset size:", len(cleaned_training_dataset))
else:
    print("No mismatches found or mismatch data is empty. No filtering applied.")
    cleaned_training_dataset = squad_raw['train']


Total number of mismatches: 1409
Original dataset size: 130319
Cleaned dataset size: 128919


In [32]:

# Preprocessing the datasets
train_dataset = cleaned_training_dataset.map(
                preprocess_function,
                batched=True,
                remove_columns=squad_raw["train"].column_names,
                desc="Running tokenizer on train dataset",
                load_from_cache_file=False,  # Disable caching
            )
eval_dataset = squad_raw['validation'].map(
                prepare_validation_features,
                batched=True,
                remove_columns=squad_raw["train"].column_names,
                desc="Running tokenizer on validation dataset",
                load_from_cache_file=False,  # Disable caching
            )
eval_examples =  squad_raw["validation"]


Running tokenizer on train dataset:   0%|          | 0/128919 [00:00<?, ? examples/s]

Running tokenizer on validation dataset:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [33]:
# Display the sizes of the splits to confirm
print("Train set size:", len(train_dataset))
print("Validation set size:", len(eval_dataset))
print("Validation Examples size:", len(eval_examples))

Train set size: 129018
Validation set size: 11955
Validation Examples size: 11873


## 3. Initial Tuning of RoBERTa Model


For Intial training we used the fine-tuned parameters of the model https://huggingface.co/deepset/roberta-base-squad2

In [56]:
no_answer_threshold = 0.0
model = AutoModelForQuestionAnswering.from_pretrained(pretrained_model_name)
training_args = TrainingArguments(
    output_dir=f'{normalized_model_name}-finetuned-manual',
    overwrite_output_dir = True,
    metric_for_best_model='f1',
    greater_is_better=True,
    load_best_model_at_end=True,
    save_total_limit=4, 
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to="wandb",  # Enable logging to Weights & Biases
    run_name=f"{normalized_model_name}-finetune-manual",  # Optionally set a specific run name    
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_ratio=0.2,
    num_train_epochs=3,
    lr_scheduler_type = 'linear',
)

trainer = QuestionAnsweringTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        eval_examples=eval_examples,
        tokenizer=tokenizer,
        data_collator=data_collator,
        post_process_function=post_processing_function,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

trainer.train()

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Exact,F1,Total,Hasans Exact,Hasans F1,Hasans Total,Noans Exact,Noans F1,Noans Total,Best Exact,Best Exact Thresh,Best F1,Best F1 Thresh
1,0.950700,No log,61.770403,76.795947,11873,42.442645,72.536822,5928,81.042893,81.042893,5945,61.770403,0.000000,76.795947,0.000000
2,0.686300,No log,62.140992,77.705702,11873,43.758435,74.932489,5928,80.470984,80.470984,5945,62.140992,0.000000,77.705702,0.000000
3,0.466600,No log,63.151689,79.349576,11873,43.758435,76.200661,5928,82.489487,82.489487,5945,63.151689,0.000000,79.349576,0.000000


  0%|          | 0/11873 [00:00<?, ?it/s]

  0%|          | 0/11873 [00:00<?, ?it/s]

  0%|          | 0/11873 [00:00<?, ?it/s]

TrainOutput(global_step=24192, training_loss=0.9035430052293041, metrics={'train_runtime': 5700.9929, 'train_samples_per_second': 67.892, 'train_steps_per_second': 4.243, 'total_flos': 1.0113595488169574e+17, 'train_loss': 0.9035430052293041, 'epoch': 3.0})

In [58]:
trainer.train()


Epoch,Training Loss,Validation Loss,Exact,F1,Total,Hasans Exact,Hasans F1,Hasans Total,Noans Exact,Noans F1,Noans Total,Best Exact,Best Exact Thresh,Best F1,Best F1 Thresh
1,0.555800,No log,61.517729,77.342007,11873,41.700405,73.394340,5928,81.278385,81.278385,5945,61.517729,0.000000,77.342007,0.000000
2,0.443000,No log,62.907437,78.506185,11873,42.780027,74.022256,5928,82.977292,82.977292,5945,62.907437,0.000000,78.506185,0.000000
3,0.294900,No log,61.846206,78.208596,11873,43.049933,75.821635,5928,80.588730,80.588730,5945,61.846206,0.000000,78.208596,0.000000


  0%|          | 0/11873 [00:00<?, ?it/s]

  0%|          | 0/11873 [00:00<?, ?it/s]

  0%|          | 0/11873 [00:00<?, ?it/s]

TrainOutput(global_step=24192, training_loss=0.4129388490051189, metrics={'train_runtime': 5665.864, 'train_samples_per_second': 68.313, 'train_steps_per_second': 4.27, 'total_flos': 1.0113595488169574e+17, 'train_loss': 0.4129388490051189, 'epoch': 3.0})

## 5. Models Tuning Reusable Functions

In [44]:
Sample_Size = 60000

# Load a sample portion of the dataset
subset_squad_raw = datasets.DatasetDict({
    'train': cleaned_training_dataset.shuffle(seed=42).select(range(Sample_Size)),
    'validation': full_squad_dataset['validation']
})
    
# Display the sizes of the splits to confirm
print("Train set size:", len(subset_squad_raw['train']))
print("Validation set size:", len(subset_squad_raw['validation']))
squad_raw

# Preprocessing the datasets
eval_examples =  subset_squad_raw["validation"]


Train set size: 60000
Validation set size: 11873


In [46]:
class AdvancedEarlyStoppingCallback(TrainerCallback):
    """
    A callback to stop training when either the performance falls below a certain threshold
    or if there is no improvement over a set number of epochs.
    """
    def __init__(self, metric_name, patience, threshold):
        self.metric_name = metric_name
        self.patience = patience
        self.threshold = threshold
        self.best_score = None
        self.no_improve_epochs = 0

    def on_evaluate(self, args, state, control, **kwargs):
        metric_value = kwargs['metrics'].get(self.metric_name)

        if self.best_score is None or metric_value > self.best_score:
            self.best_score = metric_value
            self.no_improve_epochs = 0
        else:
            self.no_improve_epochs += 1

        # Check if performance is below the threshold
        if metric_value < self.threshold:
            control.should_training_stop = True
            print(f"Stopping training: {self.metric_name} below threshold of {self.threshold}")

        # Check if no improvement has been seen over the allowed patience
        if self.no_improve_epochs >= self.patience:
            control.should_training_stop = True
            print(f"Stopping training: No improvement in {self.metric_name} for {self.patience} epochs")


In [48]:
# Define model initialization function
def model_init():
    return AutoModelForQuestionAnswering.from_pretrained(pretrained_model_name)

# Define train dataset initialization function
def train_dataset_init():
    return subset_squad_raw['train'].map(
                preprocess_function,
                batched=True,
                remove_columns=subset_squad_raw["train"].column_names,
                desc="Running tokenizer on train dataset",
            )

# Define validation dataset initialization function
def vald_dataset_init():
    return subset_squad_raw['validation'].map(
                prepare_validation_features,
                batched=True,
                remove_columns=subset_squad_raw["train"].column_names,
                desc="Running tokenizer on validation dataset",
            )

# Optuna objective function for hyperparameter tuning
def objective(trial):
    # Hyperparameters to tune 
    global no_answer_threshold
    no_answer_threshold = trial.suggest_float('no_answer_threshold', 0.0, 1.0)
    #global global_doc_stride
    #global_doc_stride=trial.suggest_int('doc_stride', 128, 256, step=64)

    learning_rate = trial.suggest_float('learning_rate', 1e-7, 1e-4, log=True)
    #batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    #warmup_steps = trial.suggest_int('warmup_steps', 0, 1000)
    warmup_ratio= trial.suggest_int('warmup_ratio', 0.0, 1.0)
    weight_decay = trial.suggest_float('weight_decay', 0.0, 0.25)
    adam_beta1 = trial.suggest_float('adam_beta1', 0.8, 0.95)
    adam_beta2 = trial.suggest_float('adam_beta2', 0.990, 0.999)
    adam_epsilon = trial.suggest_float('adam_epsilon', 1e-8, 1e-6)
    lr_scheduler_type = trial.suggest_categorical('lr_scheduler_type', ['linear', 'cosine', 'cosine_with_restarts']) #,'constant_with_warmup'
    output_dir = f"./{normalized_model_name}-finetuned-squadv2/trial_{trial.number}"
    
    # Print trial parameters
    print(f"Current Trial {trial.number} parameters: {trial.params}")
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir = True,
        metric_for_best_model='f1',
        greater_is_better=True,
        load_best_model_at_end=True,
        save_total_limit=2, # Save only the best model unless you specify a different number
        eval_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=3,  # Adjust based on computation limits
        report_to="wandb",  # Enable logging to Weights & Biases        
        run_name=f"{normalized_model_name}-finetune-squadv2",  # Optionally set a specific run name    
        learning_rate=learning_rate,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        #warmup_steps=warmup_steps,
        warmup_ratio=warmup_ratio,
        weight_decay=weight_decay,
        adam_beta1=adam_beta1,
        adam_beta2=adam_beta2,
        adam_epsilon=adam_epsilon,
        lr_scheduler_type=lr_scheduler_type,
        fp16=True,  # Enable mixed-precision training
    )    

    trainer = QuestionAnsweringTrainer(
        model=model_init(),
        tokenizer=tokenizer,
        args=training_args,
        train_dataset=train_dataset_init(),
        eval_dataset=vald_dataset_init(),
        eval_examples=eval_examples,        
        data_collator=data_collator,
        post_process_function=post_processing_function,
        compute_metrics=compute_metrics,
        callbacks=[AdvancedEarlyStoppingCallback(metric_name='eval_f1', patience=1, threshold=75)]
    )  
    

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()
    #print("Evaluation results:", eval_results)  # Debug print
    return eval_results['eval_f1']


## 6. Hyperparameters Search for RoBERTa Model


In [53]:
# Create a study object and optimize the objective
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=15)

[I 2024-05-30 23:28:45,612] A new study created in memory with name: no-name-98b12afb-c7c0-4115-bbfb-81f2f4e091de


Current Trial 0 parameters: {'no_answer_threshold': 0.6258009157706566, 'learning_rate': 5.4396651133623655e-06, 'warmup_ratio': 1, 'weight_decay': 0.05097796400038285, 'adam_beta1': 0.8659990504824278, 'adam_beta2': 0.9928074847405066, 'adam_epsilon': 6.163387498504161e-07, 'lr_scheduler_type': 'cosine'}


Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Exact,F1,Total,Hasans Exact,Hasans F1,Hasans Total,Noans Exact,Noans F1,Noans Total,Best Exact,Best Exact Thresh,Best F1,Best F1 Thresh
1,2.025800,No log,26.918218,42.066367,11873,28.036437,58.376176,5928,25.803196,25.803196,5945,50.088436,0.000000,50.103409,0.000000


  0%|          | 0/11873 [00:00<?, ?it/s]

...

  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 75


  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 75
Stopping training: No improvement in eval_f1 for 1 epochs


[I 2024-05-31 07:59:23,768] Trial 14 finished with value: 72.04099726940419 and parameters: {'no_answer_threshold': 0.19202613184778528, 'learning_rate': 6.726529288020318e-05, 'warmup_ratio': 0, 'weight_decay': 0.1153804510722297, 'adam_beta1': 0.8408079922549465, 'adam_beta2': 0.9918352290126685, 'adam_epsilon': 8.008158107345258e-07, 'lr_scheduler_type': 'cosine_with_restarts'}. Best is trial 11 with value: 74.76637427549916.


In [60]:
study.optimize(objective, n_trials=5)

Current Trial 15 parameters: {'no_answer_threshold': 0.4088604665366379, 'learning_rate': 1.7919353679520017e-05, 'warmup_ratio': 0, 'weight_decay': 0.02263510829303961, 'adam_beta1': 0.8857262944005789, 'adam_beta2': 0.9939730599299187, 'adam_epsilon': 3.3401342130278313e-07, 'lr_scheduler_type': 'cosine_with_restarts'}


Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Exact,F1,Total,Hasans Exact,Hasans F1,Hasans Total,Noans Exact,Noans F1,Noans Total,Best Exact,Best Exact Thresh,Best F1,Best F1 Thresh
1,1.083300,No log,57.803420,73.630823,11873,41.683536,73.383731,5928,73.877208,73.877208,5945,57.803420,0.000000,73.630823,0.000000


  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 75


...

  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 75


  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 75
Stopping training: No improvement in eval_f1 for 1 epochs


[I 2024-05-31 15:23:56,004] Trial 19 finished with value: 66.87037525474061 and parameters: {'no_answer_threshold': 0.014272556119162404, 'learning_rate': 4.594694463293726e-06, 'warmup_ratio': 0, 'weight_decay': 0.09825720809015176, 'adam_beta1': 0.9017057358176097, 'adam_beta2': 0.994796995372801, 'adam_epsilon': 4.4922173865234555e-07, 'lr_scheduler_type': 'cosine'}. Best is trial 11 with value: 74.76637427549916.


In [62]:
study.optimize(objective, n_trials=5)

Current Trial 20 parameters: {'no_answer_threshold': 0.3291102525100162, 'learning_rate': 2.7852612090447016e-05, 'warmup_ratio': 1, 'weight_decay': 0.005197294914123355, 'adam_beta1': 0.856679803017277, 'adam_beta2': 0.9933620995629278, 'adam_epsilon': 6.402553919783056e-07, 'lr_scheduler_type': 'cosine_with_restarts'}


Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Exact,F1,Total,Hasans Exact,Hasans F1,Hasans Total,Noans Exact,Noans F1,Noans Total,Best Exact,Best Exact Thresh,Best F1,Best F1 Thresh
1,1.375300,No log,48.151268,64.229526,11873,39.136302,71.338927,5928,57.140454,57.140454,5945,50.096858,0.000000,64.237948,0.000000


  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 75


...

  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 75


  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 75
Stopping training: No improvement in eval_f1 for 1 epochs


[I 2024-05-31 18:13:05,575] Trial 24 finished with value: 71.7581190782017 and parameters: {'no_answer_threshold': 0.21994324796805112, 'learning_rate': 1.2884647654409051e-05, 'warmup_ratio': 0, 'weight_decay': 0.06507294652050945, 'adam_beta1': 0.8310556580590547, 'adam_beta2': 0.9970572388423093, 'adam_epsilon': 7.388398049919757e-07, 'lr_scheduler_type': 'cosine_with_restarts'}. Best is trial 11 with value: 74.76637427549916.


In [64]:
study.optimize(objective, n_trials=5)

Current Trial 25 parameters: {'no_answer_threshold': 0.08985460181677338, 'learning_rate': 4.259414610191931e-05, 'warmup_ratio': 0, 'weight_decay': 0.0897817685970829, 'adam_beta1': 0.8006925908420763, 'adam_beta2': 0.9981304761105283, 'adam_epsilon': 8.445066508061142e-07, 'lr_scheduler_type': 'cosine_with_restarts'}


Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Exact,F1,Total,Hasans Exact,Hasans F1,Hasans Total,Noans Exact,Noans F1,Noans Total,Best Exact,Best Exact Thresh,Best F1,Best F1 Thresh
1,1.047200,No log,58.603554,73.908218,11873,41.885965,72.539181,5928,75.273339,75.273339,5945,58.603554,0.000000,73.908218,0.000000


  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 75


...

  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 75


  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 75
Stopping training: No improvement in eval_f1 for 1 epochs


[I 2024-05-31 21:41:42,482] Trial 29 finished with value: 72.20837608388447 and parameters: {'no_answer_threshold': 0.37015756730215865, 'learning_rate': 6.392849127783965e-05, 'warmup_ratio': 1, 'weight_decay': 0.05218544844120233, 'adam_beta1': 0.8300236564801236, 'adam_beta2': 0.9953217559810993, 'adam_epsilon': 5.687400203578246e-07, 'lr_scheduler_type': 'linear'}. Best is trial 11 with value: 74.76637427549916.


## 7. Summary of Hyperparameters Search

Each row of the below table corresponds to a trial and shows all specified parameters along with measured performance metrics with Trial 9 acheiving the highest performance.



| Trial | Learning Rate     | Batch Size | Warmup Steps | Weight Decay   | Adam Beta1 | Adam Beta2 | Adam Epsilon | LR Scheduler Type          | Exact   | F1       | Hasans Exact | Hasans F1 | Noans Exact | Noans F1 |
|-------|-------------------|------------|--------------|----------------|------------|------------|--------------|----------------------------|---------|----------|--------------|-----------|-------------|----------|


## Tuning RoBERTa Using the Best Hyperparameters

In [67]:
no_answer_threshold = 0.02889025697096306

model = AutoModelForQuestionAnswering.from_pretrained(pretrained_model_name)
training_args = TrainingArguments(
    output_dir=f"./{normalized_model_name}-best_model",
    overwrite_output_dir = True,
    metric_for_best_model='f1',
    greater_is_better=True,
    load_best_model_at_end=True,
    save_total_limit=4, 
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    report_to="wandb",  # Enable logging to Weights & Biases
    run_name=f"{normalized_model_name}-best_model",
    learning_rate=2.1932010420330285e-05,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_ratio=0,
    weight_decay=0.007873328004520265,
    adam_beta1=0.8005808315798295,
    adam_beta2=0.9989500255906361,
    adam_epsilon=9.408072279198825e-07,
    lr_scheduler_type='cosine_with_restarts',
    fp16=True,  # Enable mixed-precision training
)

trainer = QuestionAnsweringTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        eval_examples=eval_examples,
        tokenizer=tokenizer,
        data_collator=data_collator,
        post_process_function=post_processing_function,
        compute_metrics=compute_metrics,
        callbacks=[AdvancedEarlyStoppingCallback(metric_name='eval_f1', patience=3, threshold=70)]
)

trainer.train()

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Exact,F1,Total,Hasans Exact,Hasans F1,Hasans Total,Noans Exact,Noans F1,Noans Total,Best Exact,Best Exact Thresh,Best F1,Best F1 Thresh
1,0.919900,No log,63.783374,78.198175,11873,40.384615,69.255556,5928,87.115223,87.115223,5945,63.783374,0.000000,78.198175,0.000000
2,0.745600,No log,62.983239,78.092215,11873,41.143725,71.405004,5928,84.760303,84.760303,5945,62.983239,0.000000,78.092215,0.000000
3,0.555800,No log,63.555967,78.997628,11873,41.514845,72.442449,5928,85.534062,85.534062,5945,63.555967,0.000000,78.997628,0.000000
4,0.456500,No log,63.311716,79.031498,11873,42.948718,74.433362,5928,83.616484,83.616484,5945,63.311716,0.000000,79.031498,0.000000
5,0.333800,No log,62.376821,78.434245,11873,43.758435,75.919331,5928,80.941968,80.941968,5945,62.376821,0.000000,78.434245,0.000000
6,0.261200,No log,61.736714,78.058323,11873,44.163293,76.853317,5928,79.259882,79.259882,5945,61.736714,0.000000,78.058323,0.000000
7,0.200900,No log,61.475617,77.998413,11873,43.437922,76.530897,5928,79.461733,79.461733,5945,61.475617,0.000000,77.998413,0.000000


  0%|          | 0/11873 [00:00<?, ?it/s]

  0%|          | 0/11873 [00:00<?, ?it/s]

  0%|          | 0/11873 [00:00<?, ?it/s]

...

  0%|          | 0/11873 [00:00<?, ?it/s]

  0%|          | 0/11873 [00:00<?, ?it/s]

  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: No improvement in eval_f1 for 3 epochs


TrainOutput(global_step=56448, training_loss=0.5255873316810244, metrics={'train_runtime': 13342.2004, 'train_samples_per_second': 96.699, 'train_steps_per_second': 6.044, 'total_flos': 2.3598389472395674e+17, 'train_loss': 0.5255873316810244, 'epoch': 7.0})